# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_data = pd.read_csv("../WeatherPy/output/weather_data.csv")
weather_data

,name,lat,long,max_temp,humidity,cloudiness,wind_speed,country
0,Ruska Poliana,49.4170,31.9201,84.24,42,11,6.44,UA
1,Jieshi,22.8134,115.8257,86.95,80,78,14.18,CN
2,Lebu,-37.6167,-73.6500,47.37,67,1,21.88,CL
3,Pueblo Nuevo,23.3833,-105.3833,67.32,58,74,0.45,MX
4,Cidreira,-30.1811,-50.2056,54.72,64,62,19.39,BR
...,...,...,...,...,...,...,...,...
584,Ascension,30.2169,-90.9001,90.00,80,1,0.00,US
585,Senhor do Bonfim,-10.4614,-40.1894,76.12,64,82,8.57,BR
586,Gao,16.6362,1.6370,95.23,39,100,14.07,ML
587,Daye,30.0833,114.9500,78.04,93,31,8.68,CN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
locations = weather_data[["lat", "long"]]
weight = weather_data["humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
ideal_df = weather_data[((weather_data["max_temp"]>65)&(weather_data["max_temp"]<85))&(weather_data["wind_speed"]<10)&(weather_data["cloudiness"]<30)]

ideal_df

,name,lat,long,max_temp,humidity,cloudiness,wind_speed,country
0,Ruska Poliana,49.4170,31.9201,84.24,42,11,6.44,UA
7,Bemidji,47.4736,-94.8803,72.55,83,1,6.91,US
10,Konëvo,62.1215,39.3192,69.26,38,6,6.35,RU
13,Padang,-0.9492,100.3543,80.65,87,23,3.33,ID
16,Albany,42.6001,-73.9662,81.63,66,4,2.17,US
...,...,...,...,...,...,...,...,...
539,Channahon,41.4295,-88.2287,83.84,70,1,0.00,US
544,Pampa,35.5362,-100.9599,84.24,65,1,9.22,US
549,Moscow,55.7522,37.6156,84.43,27,3,5.10,RU
567,Krasnyy Kut,50.9500,46.9667,80.74,32,11,8.39,RU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
hotel_df = ideal_df
hotel_df = hotel_df.dropna(how='any')

hotel_df

,name,lat,long,max_temp,humidity,cloudiness,wind_speed,country
0,Ruska Poliana,49.4170,31.9201,84.24,42,11,6.44,UA
7,Bemidji,47.4736,-94.8803,72.55,83,1,6.91,US
10,Konëvo,62.1215,39.3192,69.26,38,6,6.35,RU
13,Padang,-0.9492,100.3543,80.65,87,23,3.33,ID
16,Albany,42.6001,-73.9662,81.63,66,4,2.17,US
...,...,...,...,...,...,...,...,...
539,Channahon,41.4295,-88.2287,83.84,70,1,0.00,US
544,Pampa,35.5362,-100.9599,84.24,65,1,9.22,US
549,Moscow,55.7522,37.6156,84.43,27,3,5.10,RU
567,Krasnyy Kut,50.9500,46.9667,80.74,32,11,8.39,RU


In [58]:



base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

count_tot = hotel_df["lat"].count()
count=0
hotels = []

while count < count_tot:

    lat = hotel_df.iloc[count, 1]
    long = hotel_df.iloc[count, 2]
    latlong = str(lat) + "," + str(long)
    
    url=f"{base_url}key={g_key}&location={latlong}&radius=5000&type=lodging"
    geo_data = requests.get(url).json()    
    
    try:
        hotel = geo_data["results"][0]["name"]
        hotels.append(hotel)
        count = count+1
        print(f"Searching City: {hotel_df.iloc[count, 0]}")
    except IndexError:
        hotel = "Skipped"
        hotels.append(hotel)
        count = count+1
        print("City not found")

print("data retrieved")
del hotels[68]
hotel_df["Hotel Name"] = hotels

City not found
Searching City: Konëvo
City not found
Searching City: Albany
City not found
Searching City: George Town
Searching City: Stony Mountain
Searching City: Alvand
Searching City: Saint Anthony
Searching City: Wausau
Searching City: Kuusamo
Searching City: Mahenge
Searching City: Saint George
Searching City: Pacific Grove
Searching City: Ponta do Sol
Searching City: Roald
Searching City: Amapá
City not found
Searching City: Songkhla
Searching City: Katobu
Searching City: Marquette
Searching City: Kushima
Searching City: Kruisfontein
Searching City: Northampton
Searching City: Miles City
Searching City: Port Elizabeth
Searching City: Nantucket
Searching City: Bengkulu
Searching City: Hayden
Searching City: Seoul
Searching City: Butterworth
Searching City: Celestún
Searching City: Aitape
City not found
Searching City: Hornepayne
Searching City: Palmer
Searching City: Huejuquilla el Alto
Searching City: Riberalta
Searching City: Bulawayo
Searching City: Pimentel
Searching City: W

/Users/ashtonsmith/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [62]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!="Skipped"]
hotel_df = hotel_df.rename(columns={"name":"City","country":"Country"})
hotel_df

,City,lat,long,max_temp,humidity,cloudiness,wind_speed,Country,Hotel Name
7,Bemidji,47.4736,-94.8803,72.55,83,1,6.91,US,Hampton Inn & Suites Bemidji
13,Padang,-0.9492,100.3543,80.65,87,23,3.33,ID,Mercure Padang
31,Jamestown,42.0970,-79.2353,82.18,63,1,8.99,US,Hampton Inn & Suites Jamestown
45,George Town,5.4112,100.3354,84.88,82,20,2.30,MY,Cititel Penang
51,Stony Mountain,50.0864,-97.2196,73.78,64,1,0.00,CA,Mountain Motor Inn
...,...,...,...,...,...,...,...,...,...
539,Channahon,41.4295,-88.2287,83.84,70,1,0.00,US,Manor Motel
544,Pampa,35.5362,-100.9599,84.24,65,1,9.22,US,"Holiday Inn Express & Suites Pampa, an IHG Hotel"
549,Moscow,55.7522,37.6156,84.43,27,3,5.10,RU,"The Ritz-Carlton, Moscow"
567,Krasnyy Kut,50.9500,46.9667,80.74,32,11,8.39,RU,Yubileynaya


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [70]:
# Add marker layer ontop of heat map
weight2 = hotel_df["humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
fig.add_layer(heat_layer)

# Display figure
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))